In [15]:
import kfp
from kfp import dsl
from kfp.components import create_component_from_func, InputPath, OutputPath
from kubernetes import client as k8s

In [16]:
client = kfp.Client()

In [3]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2023, 11, 7, 3, 51, 16, tzinfo=tzlocal()),
                  'description': '231107_Park_Test',
                  'id': 'acbdfaad-d14a-46ef-82fc-2f5715f8d1c7',
                  'name': '231107_Park_Test',
                  'resource_references': [{'key': {'id': 'modeltest',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2023, 11, 8, 20, 10, 59, tzinfo=tzlocal()),
                  'description': 'first experiment',
                  'id': '498323bc-fe83-44a4-bf7b-4215abcd539e',
                  'name': 'lab0',
                  'resource_references': [{'key': {'id': 'modeltest',
                                                   'type': 'NAMESPACE'},
                        

In [26]:
def repeat_line(output_dataframe_path: OutputPath()):
    import pandas as pd
    from pathlib import Path
    
    data = {
    'year': [2016, 2017, 2018],
    'GDP rate': [2.8, 3.1, 3.0],
    'GDP': ['1.637M', '1.73M', '1.83M']
    }
 
    df = pd.DataFrame(data)
    
    # Create the directory if it doesn't exist
    output_path = Path(output_dataframe_path)
    output_path.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a CSV file
    df.to_csv(output_path / "filename.csv", mode='w')
    
# Reading bigger data
def print_text(dataframe_path: InputPath()): # The "text" input is untyped so that any data can be printed
    import pandas as pd
    from pathlib import Path
    
    dataframe_path = Path(dataframe_path)

    # Read the DataFrame from the CSV file
    df = pd.read_csv(dataframe_path / "filename.csv")
    
    print(df)
    
    
# component 생성 시 base image 지정할 수 있음
BASE_IMAGE = "python:3.7"
first_op = create_component_from_func(repeat_line, 
                                    base_image=BASE_IMAGE,
                                      packages_to_install=["pandas", "pathlib"])

second_op = create_component_from_func(print_text, 
                                    base_image=BASE_IMAGE,
                                       packages_to_install=["pandas", "pathlib"])


# pod setting
def pod_defaults(op):
    op.set_memory_request('10Mi').set_cpu_request('10m')
    return op

In [27]:
# 2. Build a pipeline using the component                      

# pipeline의 이름은 Calculation pipeline
@dsl.pipeline(
   name='data-test',
   description='A toy pipeline'
)
def print_repeating_lines_pipeline():
    repeat_lines_task = pod_defaults(first_op())
    pod_defaults(second_op(repeat_lines_task.output))

In [28]:
arguments = {}
EXPERIMENT_NAME = 'print_data_pipeline'
kfp.Client().create_run_from_pipeline_func(print_repeating_lines_pipeline,
                                           arguments=arguments,
                                           experiment_name=EXPERIMENT_NAME)

RunPipelineResult(run_id=66f02cf6-0c08-4bd7-b652-eb0f3fec0d83)